# Method Chaining with NumPy and Intro to Pandas

v ekc

In [2]:
# Whenever you want to use numpy import it with the following code
import numpy as np

### Method chaining

In [3]:
arr2d = np.random.randint(0,20,(5,4))
arr2d

array([[ 2,  4, 15,  6],
       [ 5, 11,  1, 17],
       [ 5,  4, 16, 10],
       [ 7,  0, 19, 11],
       [ 1,  2, 17,  2]])

In [4]:
# start by reshaping
arr2d.reshape((2,10))

array([[ 2,  4, 15,  6,  5, 11,  1, 17,  5,  4],
       [16, 10,  7,  0, 19, 11,  1,  2, 17,  2]])

In [5]:
# chain: reshape and then take the max of each column
arr2d.reshape((2,10)).max(axis=0)

array([16, 10, 15,  6, 19, 11,  1, 17, 17,  4])

In [6]:
# chain: reshape and then take the max of each column then get the average of those
arr2d.reshape((2,10)).max(axis=0).mean()

np.float64(11.6)

In [7]:
# when your chains start getting long 
(arr2d.reshape((2,10))
 .max(axis=0)
 .mean())

np.float64(11.6)

In [8]:
# reminder that this doesn't change the original array
arr2d

array([[ 2,  4, 15,  6],
       [ 5, 11,  1, 17],
       [ 5,  4, 16, 10],
       [ 7,  0, 19, 11],
       [ 1,  2, 17,  2]])

### Example

I have a recipe dataset that I want to summarize!

I want to create a table of the mean calories of each recipe category.

And I want the recipe categories sorted by the average rating!

In [9]:
import pandas as pd
import ast

recipes = pd.read_csv("recipes_time_int.csv")
print(recipes.head(3))

   recipe_id                          name  \
0       6770         Pumpkin Coconut Bread   
1       6672              Strawberry Bread   
2     234425  How to Make Cronuts, Part II   

                                         description  \
0  This pumpkin coconut bread is a wonderful brea...   
1  This strawberry bread with frozen strawberries...   
2  Part I of this series showed you how to make t...   

                  date_published  rating_value  rating_count cook_time  \
0  1998-07-06T00:27:23.000-04:00           4.7          81.0     PT60M   
1  1998-05-07T15:22:52.000-04:00           4.0          49.0     PT60M   
2  2013-09-09T15:45:17.000-04:00           4.9          13.0     PT20M   

                                           nutrition prep_time total_time  \
0  {'calories': '296 kcal', 'carbohydrateContent'...     PT10M      PT70M   
1  {'calories': '551 kcal', 'carbohydrateContent'...     PT15M      PT75M   
2                                                 {}     PT60M

In [10]:

# convert nutrition_dict from string -> real actual and literal dict
recipes["nutrition_dict"] = recipes["nutrition_dict"].apply(ast.literal_eval)

# get calories
#   calories are hidden in the column 'nutrition_dict' as a key in the dictionary
#   have to account if key 'calories' is missing so we use
#          dict.get(key)
recipes["calories"] = [d.get("calories") for d in recipes["nutrition_dict"]]

# make sure the ratings and calories are numeric
recipes["rating_value"] = pd.to_numeric(recipes["rating_value"], errors="coerce")
recipes["calories"] = pd.to_numeric(recipes["calories"], errors="coerce")

# summary: mean calories + mean rating by tax1, sorted by mean rating
tax1_summary = (
    recipes
      .dropna(subset=["tax1", "rating_value", "calories"])
      .groupby("tax1")
      .agg(                       # collect summary statistics: "mean", "size"
          mean_calories=("calories", "mean"),
          mean_rating=("rating_value", "mean"),
          n_recipes=("recipe_id", "size")
      )
      .sort_values("mean_rating", ascending=False)
)

tax1_summary


,mean_calories,mean_rating,n_recipes
tax1,,,
Ingredients,38.352941,4.758824,17
HolidaysandEvents,632.648649,4.732432,37
TrustedBrands,450.250000,4.725000,12
USRecipes,82.500000,4.700000,2
BBQandGrilling,296.400000,4.688571,35
FruitsVegetablesandOtherProduce,257.909091,4.618182,33
SoupsStewsandChili,333.664459,4.600442,453
PastaandNoodles,541.304348,4.591304,23
SaladRecipes,296.017370,4.577667,403


#### No chaining

In [11]:
# get calories from dictionary
recipes["nutrition_dict"] = recipes["nutrition_dict"].apply(ast.literal_eval)

# get calories
#   calories are hidden in the column 'nutrition_dict' as a key in the dictionary
#   have to account if key 'calories' is missing so we use
#          dict.get(key)
recipes["calories"] = [d.get("calories") for d in recipes["nutrition_dict"]]

# convert col to numeric
recipes["rating_value"] = pd.to_numeric(recipes["rating_value"], errors="coerce")
recipes["calories"] = pd.to_numeric(recipes["calories"], errors="coerce")

# remove NA
cleaned = recipes.dropna(subset=["tax1", "rating_value", "calories"])

# make groups
grouped = cleaned.groupby("tax1")

# get aggregate summary: mean, size
tax1_summary = grouped.agg(
    mean_calories=("calories", "mean"),
    mean_rating=("rating_value", "mean"),
    n_recipes=("recipe_id", "size")
)

# sort by mean rating
tax1_summary = tax1_summary.sort_values("mean_rating", ascending=False)

tax1_summary

ValueError: malformed node or string: {'calories': 296, 'carbohydrateContent': 43, 'cholesterolContent': 31, 'fiberContent': 2, 'proteinContent': 4, 'saturatedFatContent': 3, 'sodiumContent': 365, 'sugarContent': 28, 'fatContent': 13, 'unsaturatedFatContent': 0}

## Activity

There are two arrays below:

`restaurant_items` is a 2D array:
- Row 1: appetizers
- Row 2: main dishes
- Row 3: beverages

`prices` is a 2D array of the same shape as `restaurant_items`. It contains the prices of each corresponding resturant item. 

Use these to answer the following questions.

In [ ]:
restaurant_items = np.array([['fries','salad','soup'],
                  ['pizza','pasta','burger'],
                  ['soda','iced tea','lemonade']])
prices = np.array([[6.50,7.25,4.75],
                  [9.50,10.25,10.75],
                  [2,2.25,3]])

### Use NumPy methods to determine how much would someone pay for their meal if they ordered the most expensive appetizer, main dish, and beverage? 

#### Answer

In [ ]:
prices.max(axis=1).sum()

### Use NumPy methods and slicing to create a 1d array containing all the resturaunt items sorted by most expensive to least expensive. 

#### Answer

In [ ]:
restaurant_items.ravel()[prices.ravel().argsort(axis=None)[::-1]]

3. Use NumPy methods and slicing to create a 1d array containing the most expensive appetizer, the most expensive main dish, and the most expensive beverage. 

In [ ]:
restaurant_items[[0,1,2],prices.argmax(axis=1)]

## Pandas
![panda](pandas.jpeg)

![series](pd_series.png)

In [ ]:
# whenever we want to use Pandas
import pandas as pd

### Creating Pandas Series

We can create a series (a 1D column) from an array, list, tuple, dictionary, and probably more!

Something fun...we can name the indicies!

In [ ]:
# Create a series from array
arr_series = pd.Series(arr2d.ravel()[:5])
arr_series

In [ ]:
# Create a series from list
lst = ['data','science','math']
lst_series = pd.Series(lst)
lst_series

In [ ]:
# Create a series from tuple
tup = (2,3,5)
tup_series = pd.Series(tup)
tup_series

In [ ]:
# Create a series from dict (keys become index, values become data)
dct = {"one": 1, "two": 2, "three": 3}
dct_series = pd.Series(dct)
dct_series

### Naming the indices: ```pd.Series(data, index = [list of index names])```

You can do it when you create the series or after the fact using the attribute ```series.index = [list of index names]```

In [ ]:
# Specify indicies when creating a series
tup_series_w_index = pd.Series(tup, index = ['two','three','five'])
tup_series_w_index

In [ ]:
# Specify indicies after
tup_series.index = ['two','three','five']
tup_series

In [ ]:
# Can also name your series
tup_series_w_index_name = pd.Series(tup, index = ['two','three','five'],name='nums')
tup_series_w_index_name

### Series attributes

In [ ]:
arr_series.dtype

In [ ]:
arr_series.shape

In [ ]:
arr_series.index

In [ ]:
tup_series_w_index_name.index

In [ ]:
arr_series.values

In [ ]:
tup_series_w_index_name.name

### Accessing Elements

You can access index with the bracket notation or with the ```series.index_name``` notation.

You can also slice with the index name or the index value....things can get weird if your index names are numbers :S

In [ ]:
dct_series

In [ ]:
# Access elements by their index (bracket notation)
dct_series['one']

In [ ]:
# Access elements by their index (dot/attribute notation)
dct_series.one

In [ ]:
# Accessing elements by their position
dct_series[0]

In [ ]:
# Slicing by index (inclusive stop)
dct_series['one':'three']

In [ ]:
# Slicing by position (exclusive stop)
dct_series[0:2]

In [ ]:
# These different ways of accessing elements can get confusing if indices are ints
new_series = pd.Series({1:1,2:2,3:3,4:4,5:5})
new_series

In [ ]:
# Are we accessing by index or position here?
new_series[1]

In [ ]:
# Are we slicing by index or position here?
new_series[1:3]

### Be explicit with `.loc` (for index-based access) and `.iloc` (for position-based access)

In [ ]:
# Access by index
new_series.loc[1]

In [ ]:
# Access by position
new_series.iloc[1]

In [ ]:
# Slice by index
new_series.loc[1:3]

In [ ]:
# Slice by position
new_series.iloc[1:3]

### Pandas dataframes

In [ ]:
my_dict = {'fruit':['apple','banana','orange'],
          'color':['red','yellow','orange'],
          'yum score':[5,5,5],
          'in fridge':[True, False, True],
          'number':[3,4,0]}

In [ ]:
fruit_df = pd.DataFrame(my_dict)
fruit_df

#### Make the fruit names the index?

##### Answer

In [ ]:
fruit_df.index = fruit_df.fruit

#### Ok, now let's just drop the ```fruit``` column :)

In [ ]:
fruit_df.drop('fruit', axis=1)

# fruit_df

In [ ]:
fruit_df.drop('fruit', axis=1, inplace=True)

print(fruit_df)

In [ ]:
list_of_tups = [(i,i**2,i**3) for i in range(10)]
list_of_tups

In [ ]:
squares_and_cubes = pd.DataFrame(list_of_tups,columns = ['n','squared','cubed'])
squares_and_cubes

##### Dataframe attributes

In [ ]:
fruit_df.dtypes

In [ ]:
fruit_df.shape

In [ ]:
fruit_df.index

In [ ]:
fruit_df.values

In [ ]:
fruit_df.columns